## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [3]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        #一个隐藏层的ReLU神经网络
        self.W1 = tf.Variable(tf.random.normal([784, 128], stddev=0.1))
        self.b1 = tf.Variable(tf.zeros([128]))

        self.W2 = tf.Variable(tf.random.normal([128, 10], stddev=0.1))
        self.b2 = tf.Variable(tf.zeros([10]))
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = tf.reshape(x, [-1, 784])
        h1 = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)
        logits = tf.matmul(h1, self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [4]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [5]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.4909747 ; accuracy 0.1024
epoch 1 : loss 2.4706593 ; accuracy 0.10455
epoch 2 : loss 2.4514265 ; accuracy 0.10643333
epoch 3 : loss 2.433157 ; accuracy 0.10873333
epoch 4 : loss 2.4157531 ; accuracy 0.11071666
epoch 5 : loss 2.3991334 ; accuracy 0.113416664
epoch 6 : loss 2.3832228 ; accuracy 0.11661667
epoch 7 : loss 2.367953 ; accuracy 0.119383335
epoch 8 : loss 2.353273 ; accuracy 0.12228333
epoch 9 : loss 2.3391314 ; accuracy 0.12548333
epoch 10 : loss 2.3254838 ; accuracy 0.1291
epoch 11 : loss 2.3122907 ; accuracy 0.13268334
epoch 12 : loss 2.2995152 ; accuracy 0.1358
epoch 13 : loss 2.287124 ; accuracy 0.13941666
epoch 14 : loss 2.2750893 ; accuracy 0.14325
epoch 15 : loss 2.2633843 ; accuracy 0.14753333
epoch 16 : loss 2.2519848 ; accuracy 0.1515
epoch 17 : loss 2.2408671 ; accuracy 0.15611666
epoch 18 : loss 2.2300138 ; accuracy 0.16055
epoch 19 : loss 2.219405 ; accuracy 0.16481666
epoch 20 : loss 2.2090251 ; accuracy 0.16901666
epoch 21 : loss 2.1988595 ; ac

可以发现如果仅仅使用一个隐藏层的很简单的模型的话训练50个epoch的效果并不理想